## 19. 세상에 없는 얼굴 GAN, 오토인코더

### 1. 가짜 제조 공정, 생성자

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# 생성자 모델 만들기
generator = Sequential()
generator.add(Dense(128*7*7, input_dim = 100, activation=LeakyReLU(0.2)))
# 128 : 임의로 정한 노드 수(128이 아니어도 되며, 충분한 노드 마련해주기)
# input_dim = 100 : 100차원 크기의 랜덤 벡터를 준비해 집어넣으라는 의미  ----> 어디에?
# 7*7 : 이미지의 최초 크기, 이유: 

generator.add(BatchNormalization())
generator.add(Reshape((7,7,128)))
generator.add(UpSampling2D())
# UpSampling2D : 이미지의 크기를 두 배씩 늘려준다.
# 7*7 레이어를 지나며 이미지 크기가 14*14가 되고, 

generator.add(Conv2D(64, kernel_size=5, padding='same'))
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
# 두 번째 UpSampling2D를 지나면 이미지 크기가 28*28이 된다.
# 작은 크기의 이미지를 점점 늘려 가면서 컨볼루션 층을 지나게 하는 것이 DCGAN의 특징

generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))

### 2. 진위를 가려내는 장치, 판별자

In [4]:
# 판별자 모델 만들기
# 모델 이름을 discriminator로 정하고 Sequential() 함수로 호출
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding='same'))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False

### 3. 적대적 신경망 실행하기

In [5]:
ginput = Input(shape=(100,))
dis_output = discriminator(generator(ginput)) 
gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [6]:
# 실행 함수를 선언합니다.
def gan_train(epoch, batch_size, saving_interval): # 세 가지 변수 지정

# MNIST 데이터 불러오기
    # MNIST 데이터를 다시 불러와 이용합니다. 단, 테스트 과정은 필요 없고
    # 이미지만 사용할 것이기 때문에 X_train만 호출합니다.
    (X_train, _), (_, _) = mnist.load_data()

    # 가로 28픽셀, 세로 28픽셀이고 흑백이므로 1을 설정합니다.
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')

    # 0~255 사이 픽셀 값에서 127.5를 뺀 후 127.5로 나누면 -1~1 사이 값으로 바뀝니다.
    X_train = (X_train - 127.5) / 127.5

In [7]:
true = np.ones((batch_size, 1))
idx = np.random.randint(0, X_train.shape[0], batch_size) 
imgs = X_train[idx]
d_loss_real = discriminator.train_on_batch(imgs, true)

NameError: name 'batch_size' is not defined

In [ ]:
fake = np.zeros((batch_size, 1)) 
noise = np.random.normal(0, 1, (batch_size, 100))
gen_imgs = generator.predict(noise)
d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)

In [8]:
# d_loss_real, d_loss_fake 값을 더해 둘로 나눈 평균이 바로 판별자의 오차
d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

NameError: name 'd_loss_real' is not defined

In [ ]:
g_loss = gan.train_on_batch(noise, true)

In [ ]:
print('epoch:%d' % i, ' d_loss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss)

### GAN 모델 만들기

In [8]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape,Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

import numpy as np
import matplotlib.pyplot as plt

# 생성자 모델을 만듭니다.
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))
generator.add(BatchNormalization())
generator.add(Reshape((7, 7, 128)))
generator.add(UpSampling2D())
generator.add(Conv2D(64, kernel_size=5, padding='same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))

# 판별자 모델을 만듭니다.
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False

# 생성자와 판별자 모델을 연결시키는 gan 모델을 만듭니다.
ginput = Input(shape=(100,))
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()

# 신경망을 실행시키는 함수를 만듭니다.
def gan_train(epoch, batch_size, saving_interval):

# MNIST 데이터 불러오기
    # 앞서 불러온 MNIST를 다시 이용합니다. 테스트 과정은 필요 없고
    # 이미지만 사용할 것이기 때문에 X_train만 호출합니다.
    (X_train, _), (_, _) = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')

    # 127.5를 뺀 후 127.5로 나누어서 -1~1 사이의 값으로 바꿉니다.
    X_train = (X_train - 127.5) / 127.5

    true = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for i in range(epoch):
        # 실제 데이터를 판별자에 입력하는 부분입니다.
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        d_loss_real = discriminator.train_on_batch(imgs, true)

        # 가상 이미지를 판별자에 입력하는 부분입니다.
        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)

        # 판별자와 생성자의 오차를 계산합니다.
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        g_loss = gan.train_on_batch(noise, true)

        print('epoch:%d' % i, ' d_loss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss)

        # 중간 과정을 이미지로 저장하는 부분입니다. 정해진 인터벌만큼 학습되면
        # 그때 만든 이미지를 gan_images 폴더에 저장하라는 의미입니다.
        # 이 코드는 이 장의 주된 목표와는 관계가 없어서 소스 코드만 소개합니다.
        if i % saving_interval == 0:
        # r, c = 5, 5
        noise = np.random.normal(0, 1, (25, 100))
        gen_imgs = generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(5, 5)
        count = 0
        for j in range(5):
            for k in range(5):
                axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                axs[j, k].axis('off')
                count += 1
        fig.savefig("gan_images/gan_mnist_%d.png" % i)

# 2000번 반복되고(+1을 하는 것에 주의),
# 배치 크기는 32, 200번마다 결과가 저장됩니다.
gan_train(2001, 32, 200)

IndentationError: expected an indented block (7345740.py, line 78)